In [1]:
# General utilities
!pip install --no-index /kaggle/input/lightglue-wheel/lightglue-0.0-py3-none-any.whl
!pip install --no-index /kaggle/input/opencv-python-wheel/opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!cp -r /kaggle/input/lightglue-weights/weights /opt/conda/lib/python3.10/site-packages/lightglue/
#!pip install --no-index /kaggle/input/pycolmap/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

import os
from tqdm import tqdm
from time import time
from fastprogress import progress_bar
import gc
import math
import numpy as np
from collections import defaultdict
from copy import deepcopy
import matplotlib.pyplot as plt
import concurrent.futures

from torchvision.transforms.functional import to_pil_image
# CV/ML
import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image

import sys
import random

    
random.seed(42)

import pycolmap

import matplotlib.pyplot as plt
from kornia.feature.adalam import AdalamFilter

# sys.path.append("/home/roy/kaggle/image-match/RoMa")
# from roma import roma_outdoor

import os, argparse, h5py, warnings
from tqdm import tqdm
from PIL import Image, ExifTags

print("Kornia version", K.__version__)
print("Pycolmap version", pycolmap.__version__)
print("Cv2 version", cv2.__version__)

#pycolmap.logging.minloglevel = 999

Processing /kaggle/input/lightglue-wheel/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/opencv-python-wheel/opencv_python-4.8.1.78-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninstalled opencv-python-4.5.4.60


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Kornia version 0.6.12
Pycolmap version 0.4.0
Cv2 version 4.8.1


In [2]:
from pathlib import Path

sys.path.append("/kaggle/input/hloc-and-weights/hloc")
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_exhaustive
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
from hloc.localize_sfm import QueryLocalizer, pose_from_cluster

In [3]:
# Mode can only be train or test. This will be used to find the image directory.
# Use "test" for submission 
MODE = "train"
MODE = "test"

# Option to change path for local testing
is_local = True
is_local = False

SPLIT = True

if is_local:
    NUM_CORES = 2
    SRC = "../input/image-matching-challenge-2024"
    MODEL_DIR = "../input/kornia-local-feature-weights/"
    DISK_PATH = "./loftr_disk.ckpt"
    HARDNET_PT = "../input/kornia-local-feature-weights/hardnet8v2.pt"
else:
    NUM_CORES = 2
    SRC = "/kaggle/input/image-matching-challenge-2024"
    MODEL_DIR = "/kaggle/input/kornia-local-feature-weights/"
    DISK_PATH = "/kaggle/input/disk/pytorch/depth-supervision/1/loftr_outdoor.ckpt"
    HARDNET_PT = "/kaggle/input/hardnet8v2/hardnet8v2.pt"

LOG_MESSAGE = "Final submission"
MATCHES_CAP = None

DEBUG = True
DEBUG = False

DEBUG_DATASET = ['dioscuri']
#'transp_obj_glass_cylinder','transp_obj_glass_cup','church','dioscuri','pond','lizard','multi-temporal-temple-baalshamin'
# Longer edge limit of the input image
hardnet_res = 1600

MODEL_DICT = {
    "Keynet": {"enable": True, "resize_long_edge_to": hardnet_res, "pair_only": False},
    "GFTT": {"enable": True, "resize_long_edge_to": hardnet_res},
    "DoG": {"enable": True, "resize_long_edge_to": hardnet_res},
    "Harris": {"enable": True, "resize_long_edge_to": hardnet_res},
    "Disk": {"enable": False},
    "Superpoint": {"enable":True},
    "RoMa": {"enable": False}
}

#Keynet    0.12
#GFTT      0.13
#DoG       0.15
#Harris    0.14

#DoG - Disk gives 0.16 in 3 hours does this transfer to CV? 
#If so do we consider more reconstruction on this?

# Find fundamental matrix parameters
FM_PARAMS = {"ransacReprojThreshold": 5, "confidence": 0.9999, "maxIters": 50000, "removeOutliers": True}

# Remove a "match" if the number of matches is lower than MATCH_FILTER_RATIO*max_num_matches
# e.g. img1 and img2 have max 10000 matches with some other images, img2 and img1 only have 99 matches. The matches btw img1 and img2 won't be selected.
MATCH_FILTER_RATIO = 0.01

# for logging
LOG_DICT = dict()
LOG_DICT["mode"] = MODE
LOG_DICT["log_message"] = LOG_MESSAGE
LOG_DICT["matches_cap"] = MATCHES_CAP
LOG_DICT["debug"] = DEBUG
LOG_DICT["debug_dataset"] = DEBUG_DATASET

if MODE == "test":
    DEBUG = False
device = torch.device("cuda")
print(torch.cuda.is_available())


True


In [4]:
# Get datadict from csv.
if MODE == "train":
    sample_path = f"{SRC}/train/train_labels.csv"
else:
    sample_path = f"{SRC}/sample_submission.csv"

data_dict = {}
with open(sample_path, "r") as f:
    for i, l in enumerate(f):
        # Skip header.
        if l and i > 0:
            if MODE == "train":
                image, _, _, _, dataset, _ = l.strip().split(",") #image_name,rotation_matrix,translation_vector,calibration_matrix,dataset,scene
            else:
                image, dataset, _, _, _ = l.strip().split(",") # image_path,dataset,scene,rotation_matrix,translation_vector
            if dataset not in data_dict:
                data_dict[dataset] = []
            if MODE == 'train':
                data_dict[dataset].append(f"{MODE}/{dataset}/images/{image}") #no more scene
            else:
                data_dict[dataset].append(image)
                

all_datasets = []
dataset_len = []
for dataset in data_dict:
    print(f"{dataset} -> {len(data_dict[dataset])} images")
    if DEBUG and (dataset not in DEBUG_DATASET):
        continue
    all_datasets.append(dataset)
    dataset_len.append(len(data_dict[dataset]))


# sort all scenes by length, lowest first
all_datasets = [x for _, x in sorted(zip(dataset_len, all_datasets), reverse=True)] #longest list first

# Print reconst order
print("Reconstruction order: ")
for dataset in all_datasets:
    print(f" --{dataset}")

church -> 41 images
Reconstruction order: 
 --church


In [5]:
def arr_to_str(a):
    return ";".join([str(x) for x in a.reshape(-1)])

# Function to create a submission file.
def create_submission(out_results, data_dict, mode="test"):
    if mode == "train":
        file_name = "submission_train.csv"
    else:
        file_name = "submission.csv"

    with open(file_name, "w") as f:
        f.write("image_path,dataset,scene,rotation_matrix,translation_vector\n")
        for dataset in data_dict:
            same_angle = False
            vectors = []
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}

            T_average = np.zeros((3))   
            T_cnt = 0
            for image in data_dict[dataset]:
                if image in res:
                    print(image)
                    R = res[image]["R"].reshape(-1)
                    T = res[image]["t"].reshape(-1)
                    # if same_angle: R, T = add_noise_if_necessary(R, T) 
                    T_average += T
                    T_cnt += 1
                else:
                    R = np.eye(3).reshape(-1)
                    T = np.zeros((3)) 
                f.write(
                    f"{image},{dataset},{dataset},{arr_to_str(R)},{arr_to_str(T)}\n"
                )

In [6]:
def load_torch_image(fname, device=torch.device("cpu")):
    img = K.image_to_tensor(cv2.imread(fname), False).float() / 255.0
    img = K.color.bgr_to_rgb(img.to(device))
    return img


def resize_torch_image(
    timg, resize_long_edge_to=None, align=None, disable_enlarge=True
):
    h, w = timg.shape[2:]
    raw_size = torch.tensor(timg.shape[2:])
    if resize_long_edge_to is None:
        scale = 1
    else:
        scale = float(resize_long_edge_to) / float(max(raw_size[0], raw_size[1]))

    if disable_enlarge:
        scale = min(scale, 1)

    h_resized = int(h * scale)
    w_resized = int(w * scale)

    if align is not None:
        assert align > 0
        h_resized = h_resized - h_resized % align
        w_resized = w_resized - w_resized % align
    scale_h = h_resized / h
    scale_w = w_resized / w

    timg_resized = K.geometry.resize(timg, (h_resized, w_resized), antialias = True)
    return timg_resized, scale_h, scale_w

def pad_to_max_size(img1, img2):
    max_h = max(img1.size(2), img2.size(2))
    max_w = max(img1.size(3), img2.size(3))
    
    # Pad to the right and the bottom to the maximum width and height
    pad_img1 = F.pad(img1, (0, max_w - img1.size(3), 0, max_h - img1.size(2)), 'constant', 0)
    pad_img2 = F.pad(img2, (0, max_w - img2.size(3), 0, max_h - img2.size(2)), 'constant', 0)
    
    return pad_img1, pad_img2



def get_roi_image(timg, roi):
    min_h = int(roi["roi_min_h"])
    min_w = int(roi["roi_min_w"])
    max_h = int(roi["roi_max_h"])
    max_w = int(roi["roi_max_w"])
    roi_img = timg[:, :, min_h:max_h, min_w:max_w]
    roi_w_scale = (max_w - min_w) / timg.shape[3]
    roi_h_scale = (max_h - min_h) / timg.shape[2]
    return roi_img, min_h, min_w

In [7]:
# Visualzation block
def draw_keypoints(img, keypoints, color=(0, 255, 0)):
    max_edge = max(img.shape[0], img.shape[1])
    good_radius = 4
    for kp in keypoints:
        x, y = kp
        cv2.circle(img, (int(x), int(y)), color=color, radius=good_radius, thickness=-1)


def draw_roi(img, roi, color=(0, 255, 255)):
    x1, y1, x2, y2 = (
        roi["roi_min_w"],
        roi["roi_min_h"],
        roi["roi_max_w"],
        roi["roi_max_h"],
    )
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color=color, thickness=2)


def plot_images_with_keypoints(fname1, fname2, kpts1, kpts2, matches, rois=None):
    print(fname1, fname2)
    # Draw keypoints on the images
    image1 = cv2.imread(fname1)
    image2 = cv2.imread(fname2)
    print(image1.shape, image2.shape)

    # draw_keypoints(image1, kpts1)
    # draw_keypoints(image2, kpts2)
    if rois is not None:
        draw_roi(image1, rois[0])
        draw_roi(image2, rois[1])
    print(image1.shape, image2.shape)
    print("Number of matches:", len(matches))
    print("Number of keypoints:", len(kpts1), len(kpts2))
    #print the first match
    print(matches[0])
    # Resize image1 and image2 to have the same smaller height
    display_h = 840
    h1, w1 = image1.shape[:2]
    h2, w2 = image2.shape[:2]
    # new_h = min(h1, h2)
    scale1 = display_h / h1
    scale2 = display_h / h2
    new_w1 = int(w1 * scale1)
    new_w2 = int(w2 * scale2)

    image1 = cv2.resize(image1, (new_w1, display_h))
    image2 = cv2.resize(image2, (new_w2, display_h))

    # Create a new image by horizontally concatenating the two images
    concatenated_img = cv2.hconcat([image1, image2])

    # Draw lines between the matching keypoints
    for match in matches:
        img1_idx = match[0]
        img2_idx = match[1]
        (x1, y1) = kpts1[img1_idx] * scale1
        (x2, y2) = kpts2[img2_idx] * scale2
        pt1 = (int(x1), int(y1))
        pt2 = (int(x2) + image1.shape[1], int(y2))
        cv2.line(concatenated_img, pt1, pt2, (0, 0, 255), 1)

    # Plot the concatenated image
    plt.figure(figsize=(20, 12))
    plt.imshow(cv2.cvtColor(concatenated_img, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()

In [8]:
# Code to manipulate a colmap database.
# Forked from https://github.com/colmap/colmap/blob/dev/scripts/python/database.py

# Copyright (c) 2018, ETH Zurich and UNC Chapel Hill.
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#
#     * Neither the name of ETH Zurich and UNC Chapel Hill nor the names of
#       its contributors may be used to endorse or promote products derived
#       from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE
# ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDERS OR CONTRIBUTORS BE
# LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR
# CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF
# SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS
# INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN
# CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.
#
# Author: Johannes L. Schoenberger (jsch-at-demuc-dot-de)

# This script is based on an original implementation by True Price.

import sys
import sqlite3


IS_PYTHON3 = sys.version_info[0] >= 3

MAX_IMAGE_ID = 2**31 - 1

CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""

CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""

CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(
    MAX_IMAGE_ID
)

CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""

CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""

CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""

CREATE_NAME_INDEX = "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"

CREATE_ALL = "; ".join(
    [
        CREATE_CAMERAS_TABLE,
        CREATE_IMAGES_TABLE,
        CREATE_KEYPOINTS_TABLE,
        CREATE_DESCRIPTORS_TABLE,
        CREATE_MATCHES_TABLE,
        CREATE_TWO_VIEW_GEOMETRIES_TABLE,
        CREATE_NAME_INDEX,
    ]
)


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


def array_to_blob(array):
    if IS_PYTHON3:
        return array.tostring()
    else:
        return np.getbuffer(array)


def blob_to_array(blob, dtype, shape=(-1,)):
    if IS_PYTHON3:
        return np.fromstring(blob, dtype=dtype).reshape(*shape)
    else:
        return np.frombuffer(blob, dtype=dtype).reshape(*shape)


class COLMAPDatabase(sqlite3.Connection):
    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)

    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = lambda: self.executescript(
            CREATE_DESCRIPTORS_TABLE
        )
        self.create_images_table = lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = lambda: self.executescript(
            CREATE_TWO_VIEW_GEOMETRIES_TABLE
        )
        self.create_keypoints_table = lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(
        self, model, width, height, params, prior_focal_length=False, camera_id=None
    ):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (
                camera_id,
                model,
                width,
                height,
                array_to_blob(params),
                prior_focal_length,
            ),
        )
        return cursor.lastrowid

    def add_image(
        self, name, camera_id, prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None
    ):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (
                image_id,
                name,
                camera_id,
                prior_q[0],
                prior_q[1],
                prior_q[2],
                prior_q[3],
                prior_t[0],
                prior_t[1],
                prior_t[2],
            ),
        )
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert len(keypoints.shape) == 2
        assert keypoints.shape[1] in [2, 4, 6]

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),),
        )

    def add_descriptors(self, image_id, descriptors):
        descriptors = np.ascontiguousarray(descriptors, np.uint8)
        self.execute(
            "INSERT INTO descriptors VALUES (?, ?, ?, ?)",
            (image_id,) + descriptors.shape + (array_to_blob(descriptors),),
        )

    def add_matches(self, image_id1, image_id2, matches):
        assert len(matches.shape) == 2
        assert matches.shape[1] == 2

        if image_id1 > image_id2:
            matches = matches[:, ::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),),
        )
        
        

    def add_two_view_geometry(
        self,
        image_id1,
        image_id2,
        matches,
        F=np.eye(3),
        E=np.eye(3),
        H=np.eye(3),
        config=2,
    ):
        assert len(matches.shape) == 2
        assert matches.shape[1] == 2

        if image_id1 > image_id2:
            matches = matches[:, ::-1]

        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,)
            + matches.shape
            + (
                array_to_blob(matches),
                config,
                array_to_blob(F),
                array_to_blob(E),
                array_to_blob(H),
            ),
        )

In [9]:
# Modified from https://github.com/cvlab-epfl/disk/blob/37f1f7e971cea3055bb5ccfc4cf28bfd643fa339/colmap/h5_to_db.py

#  Copyright [2020] [Michał Tyszkiewicz, Pascal Fua, Eduard Trulls]
#
#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at
#
#       http://www.apache.org/licenses/LICENSE-2.0
#
#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.




def get_focal(image_path, err_on_default=False):
    image = Image.open(image_path)
    max_size = max(image.size)

    exif = image.getexif()
    
    #
    # Modified to add exif_ifd to exif dict
    #
    exif_ifd = exif.get_ifd(0x8769)
    exif.update(exif_ifd)

    focal = None
    is_from_exif = False
    if exif is not None:
        focal_35mm = None
        # https://github.com/colmap/colmap/blob/d3a29e203ab69e91eda938d6e56e1c7339d62a99/src/util/bitmap.cc#L299
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == "FocalLengthIn35mmFilm":
                focal_35mm = float(value)
                is_from_exif = True
                break

        if focal_35mm is not None:
            focal = focal_35mm / 35.0 * max_size

    if focal is None:
        if err_on_default:
            raise RuntimeError("Failed to find focal length")

        # failed to find it in exif, use prior
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size
    
    #
    # Modified to return a bool indicating if the focal length is from exif 
    #
    return focal, is_from_exif


def create_camera(db, image_path, camera_model):
    image = Image.open(image_path)
    width, height = image.size

    focal, is_from_exif = get_focal(image_path)

    if camera_model == "simple-pinhole":
        model = 0  # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == "pinhole":
        model = 1  # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == "simple-radial":
        model = 2  # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == "opencv":
        model = 4  # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0.0, 0.0, 0.0, 0.0])

    #
    # Modified to set prior_focal_length if the focal length is from exif
    #
    return db.add_camera(
        model, width, height, param_arr, prior_focal_length=is_from_exif
    )


def add_kpts_matches(db, img_dir, kpts, matches, fms = None):
    fname_to_id = {}

    # Add keypoints
    for filename in tqdm(kpts):
        path = os.path.join(img_dir, filename)
        camera_model = "simple-radial"
        camera_id = create_camera(db, path, camera_model)
        image_id = db.add_image(filename, camera_id)
        fname_to_id[filename] = image_id
        db.add_keypoints(image_id, kpts[filename])

        n_keys = len(matches)
        n_total = (n_keys * (n_keys - 1)) // 2
    # Add matches
    added = set()
    with tqdm(total=n_total) as pbar:
        for key1 in matches:
            for key2 in matches[key1]:
                id_1 = fname_to_id[key1]
                id_2 = fname_to_id[key2]
                pair_id = image_ids_to_pair_id(id_1, id_2)
                if pair_id in added:
                    warnings.warn(f"Pair {pair_id} ({id_1}, {id_2}) already added!")
                    continue
                db.add_matches(id_1, id_2, matches[key1][key2])
                added.add(pair_id)
                pbar.update(1)
                if fms[key1][key2] is not None:
                    db.add_two_view_geometry(id_1, id_2, matches[key1][key2], fms[key1][key2], np.eye(3), np.eye(3))

    db.commit()


In [10]:
def get_unique_idxs(A, dim=0):
    # https://stackoverflow.com/questions/72001505/how-to-get-unique-elements-and-their-firstly-appeared-indices-of-a-pytorch-tenso
    unique, idx, counts = torch.unique(
        A, dim=dim, sorted=True, return_inverse=True, return_counts=True
    )
    _, ind_sorted = torch.sort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0], device=cum_sum.device), cum_sum[:-1]))
    first_indices = ind_sorted[cum_sum]
    return first_indices

In [11]:
# Making kornia local features loading w/o internet
class AffNetHardNet(KF.LocalFeature):
    """Convenience module, which implements KeyNet detector + AffNet + HardNet descriptor.

    .. image:: _static/img/keynet_affnet.jpg
    """

    def __init__(
        self,
        num_features: int = 5000,
        upright: bool = False,
        device=torch.device("cpu"),
        scale_laf: float = 1.0,
        detector = "keynet"
    ):
        detector_options = ["keynet", "GFTT", "Hessian", "Harris", "DoG"]
        if detector not in detector_options:
            raise ValueError("Detector must be one of {}".format(detector_options))
        
        ori_module = (
            KF.PassLAF()
            if upright
            else KF.LAFOrienter(angle_detector=KF.OriNet(False)).eval()
        )
        if not upright:
            weights = torch.load(os.path.join(MODEL_DIR, "OriNet.pth"))["state_dict"]
            ori_module.angle_detector.load_state_dict(weights)

        config = {
            # Extraction Parameters
            "nms_size": 15,
            "pyramid_levels": 5,   #4,
            "up_levels": 1,
            "scale_factor_levels": math.sqrt(2),
            "s_mult": 22.0,
        }

        if detector == "keynet":
            detector = KF.KeyNetDetector(
            False,
            num_features=num_features,
            ori_module=ori_module,
            aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
            kn_weights = torch.load(os.path.join(MODEL_DIR, "keynet_pytorch.pth"))[
            "state_dict"
            ]
            detector.model.load_state_dict(kn_weights)
        elif detector == "GFTT":
            detector = KF.MultiResolutionDetector(
                KF.CornerGFTT(),
                num_features=num_features,
                config=config,
                ori_module=ori_module,
                aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
        elif detector == "Harris":
            detector = KF.MultiResolutionDetector(
                KF.CornerHarris(0.04),
                num_features=num_features,
                config=config,
                ori_module=ori_module,
                aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
        elif detector == "DoG":
            detector = KF.MultiResolutionDetector(
                KF.BlobDoGSingle(),
                num_features=num_features,
                config=config,
                ori_module=ori_module,
                aff_module=KF.LAFAffNetShapeEstimator(False).eval(),
            ).to(device)
        affnet_weights = torch.load(os.path.join(MODEL_DIR, "AffNet.pth"))["state_dict"]
        detector.aff.load_state_dict(affnet_weights)

        # hardnet = KF.HardNet(False).eval()
        # hn_weights = torch.load(os.path.join(MODEL_DIR, "HardNetLib.pth"))["state_dict"]
        # hardnet.load_state_dict(hn_weights)
        # descriptor = KF.LAFDescriptor(
        #     hardnet, patch_size=32, grayscale_descriptor=True
        # ).to(device)
        hardnet8 = KF.HardNet8(False).eval()
        hn8_weights = torch.load(HARDNET_PT)
        hardnet8.load_state_dict(hn8_weights)
        descriptor = KF.LAFDescriptor(
            hardnet8, patch_size=32, grayscale_descriptor=True
        ).to(device)
        super().__init__(detector, descriptor, scale_laf)


In [12]:
class AffNetHardNetDetector:
    def __init__(
        self,
        model,
        device=torch.device("cuda"),
        resize_long_edge_to=600,
        matcher="adalam",
        min_matches=15,
        rgb_input = False
    ):
        self.rgb_input = rgb_input
        print("Init AffNetHardNetDetector")
        self.model = model
        self.device = device
        self.resize_long_edge_to = resize_long_edge_to
        print("Longer edge will be resized to", self.resize_long_edge_to)

    ### all the features needed of a single picture
    def detect_features(self, img_fnames):
        f_lafs = dict()
        f_descs = dict()
        f_kpts = dict()
        f_raw_size = dict()
        f_matches = dict()
        # Get features
        print("Detecting AffNetHardNet features")
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split("/")[-1]
            key = img_fname
            f_matches[key] = dict()
            with torch.inference_mode():
                timg = load_torch_image(img_path, device=device)
                raw_size = torch.tensor(timg.shape[2:])
                timg_resized, h_scale, w_scale = resize_torch_image(
                    timg, self.resize_long_edge_to, disable_enlarge=True
                )
                if self.rgb_input:
                    lafs, resps, descs = self.model(timg_resized)
                else:
                    lafs, resps, descs = self.model(K.color.rgb_to_grayscale(timg_resized))
                
                # Recover scale?
                lafs[:, :, 0, :] *= 1 / w_scale
                lafs[:, :, 1, :] *= 1 / h_scale
                desc_dim = descs.shape[-1]
                # Move keypoints to cpu for later colmap operations
                kpts = KF.get_laf_center(lafs).reshape(-1, 2).detach().cpu().numpy()
                descs = descs.reshape(-1, desc_dim).detach()
                f_lafs[key] = lafs.detach()
                f_kpts[key] = kpts
                f_descs[key] = descs
                f_raw_size[key] = raw_size
        gc.collect()
        torch.cuda.empty_cache()
        return f_lafs, f_kpts, f_descs, f_raw_size

In [13]:
def modify_lightgluematcher():
    def new_init(self, feature_name: str = "disk", params = {}):
        super(KF.LightGlueMatcher, self).__init__(feature_name)
        self.feature_name = feature_name
        self.params = params
        self.matcher = KF.LightGlue(None, **params)
        
    KF.LightGlueMatcher.__init__ = new_init

    
# def modify_disk():
#     @classmethod
#     def new_from_pretrained(cls, checkpoint: str = "depth", device: torch.device = torch.device("cpu")):
#         urls = {
#             "depth": "/home/roy/kaggle/image-match/working/weights/depth-save.pth",
#             "epipolar": "/home/roy/kaggle/image-match/working/weights/epipolar-save.pth",
#         }

#         if checkpoint not in urls:
#             raise ValueError(f"Unknown pretrained model: {checkpoint}")

#         # Load the state dictionary from the file system
#         file_path = urls[checkpoint]
#         pretrained_dict = torch.load(file_path, map_location=device)  # Ensure it loads to the right device

#         # Create an instance of the model
#         model = cls().to(device)
#         # Load the state dictionary into the model
#         model.load_state_dict(pretrained_dict["extractor"])  # Assuming the file contains a dict with key "extractor"
#         model.eval()
#         return model

#     KF.DISK.from_pretrained = new_from_pretrained

In [14]:
class LafMatcher:
    def __init__(self, min_matches=50, device="cuda", matcher="adalam"):
        self.adalam_config = KF.adalam.get_adalam_default_config()
        self.adalam_config["force_seed_mnn"] = True
        self.adalam_config["search_expansion"] = 16
        self.adalam_config["ransac_iters"] = 256
        self.adalam_config["device"] = device
        # self.adalam_config["orientation_difference_threshold"] = None
        # self.adalam_config['scale_rate_threshold'] = None
        self.min_matches = min_matches
        self.matcher = matcher

    def match(self, img_fnames, f_lafs, f_kpts, f_descs, f_raw_size, get_roi = False):
        index_pairs = dict()
        num_imgs = len(img_fnames)
        print("Matching to get index pairs")
        pair_count = 0
        f_matches = defaultdict(dict)
        f_rois = defaultdict(dict)
        for idx1 in tqdm(range(num_imgs - 1)):
            index_pairs[idx1] = []
            for idx2 in range(idx1 + 1, num_imgs):
                # if random.random() > 0.1: continue
                fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
                key1, key2 = fname1.split("/")[-1], fname2.split("/")[-1]
                lafs1 = f_lafs[key1]
                lafs2 = f_lafs[key2]
                desc1 = f_descs[key1]
                desc2 = f_descs[key2]
                hw1, hw2 = f_raw_size[key1], f_raw_size[key2]
                if self.matcher == "adalam":
                    dists, idxs = KF.match_adalam(desc1, desc2, lafs1, lafs2, hw1=hw1, hw2=hw2, config=self.adalam_config) 
                else:
                    dists, idxs = KF.match_smnn(desc1, desc2, 0.98)

                if dists.mean().cpu().numpy() < 0.5: # default 0.5
                    first_indices = get_unique_idxs(idxs[:, 1])
                    idxs = idxs[first_indices]
                    dists = dists[first_indices]
                    n_matches = len(idxs)
                    if n_matches >= self.min_matches: ## seems that for two image to overlap 1) distance < 0.5 and 2) n_match > min 
                        pair_count += 1
                        index_pairs[idx1].append(
                            [idx2, dists.mean().cpu().numpy().item(), n_matches]
                        )
                        f_matches[key1][key2] = (
                            idxs.detach().cpu().numpy().reshape(-1, 2)
                        )
                        

        print(f" Get {pair_count} from {int(num_imgs * (num_imgs-1)/2)} possible pairs")
        torch.cuda.empty_cache()
        gc.collect()
        return index_pairs, f_kpts, f_matches, f_rois
    
    
'''
index_pairs[image1][0] = [image2,mean_dist,matched_pts]
f_kpt = single_image
f_matches[image1][image2] = idx (num_match,2)
'''
    

'\nindex_pairs[image1][0] = [image2,mean_dist,matched_pts]\nf_kpt = single_image\nf_matches[image1][image2] = idx (num_match,2)\n'

In [15]:
def merge_kpts_matches(kpts, matches, new_kpts, new_matches, cap = None):
    # merge kpts
    prev_len = dict()
    for new_key in new_kpts:
        if new_key in kpts:
            old_len = len(kpts[new_key])
            kpts[new_key] = np.concatenate([kpts[new_key], new_kpts[new_key]], axis=0)
        else:
            old_len = 0
            kpts[new_key] = new_kpts[new_key]
        prev_len[new_key] = old_len

    for new_key1 in new_matches:
        for new_key2 in new_matches[new_key1]:
            old_len1 = prev_len[new_key1]
            old_len2 = prev_len[new_key2]
            new_match = new_matches[new_key1][new_key2] + [old_len1, old_len2] #offset indices shift
            if cap is not None and len(new_match) > cap:
                keep = np.random.choice(len(new_match), cap, replace=False)
                new_match = new_match[keep, :]
            if new_key1 in matches and new_key2 in matches[new_key1]:

                matches[new_key1][new_key2] = np.concatenate(
                    [
                        matches[new_key1][new_key2],
                        new_match,
                    ],
                    axis=0,
                )
            else:
                if new_key1 not in matches:
                    matches[new_key1] = dict()
                matches[new_key1][new_key2] = new_match
    return kpts, matches


def select_matches(matches, keep_ratio=0.01):
    max_matches = defaultdict(int)
    old_matches_count = 0
    for key1 in matches:
        for key2 in matches[key1]:
            max_matches[key1] = max(max_matches[key1], len(matches[key1][key2]))
            max_matches[key2] = max(max_matches[key2], len(matches[key1][key2]))
            old_matches_count += 1

    new_matches_count = 0
    new_matches = defaultdict(dict)
    for key1 in matches:
        for key2 in matches[key1]:
            n_matches = len(matches[key1][key2])
            if n_matches > max_matches[key1] * keep_ratio or n_matches > max_matches[key2] * keep_ratio:
                new_matches[key1][key2] = matches[key1][key2]
                new_matches_count += 1
    
    print(f"origin matches: {old_matches_count}, kept matches: {new_matches_count}")
    return new_matches


def select_matches(matches, keep_ratio = 0.01):
    max_matches = defaultdict(int)
    old_matches_count = 0
    for key1 in matches:
        for key2 in matches[key1]:
            max_matches[key1] = max(max_matches[key1], len(matches[key1][key2]))
            max_matches[key2] = max(max_matches[key2], len(matches[key1][key2]))
            old_matches_count +=1

    new_matches_count = 0
    new_matches = defaultdict(dict)
    for key1 in matches:
        for key2 in matches[key1]:
            n_matches = len(matches[key1][key2])
            if n_matches > max_matches[key1] * keep_ratio or n_matches > max_matches[key2] * keep_ratio:
                new_matches[key1][key2] = matches[key1][key2]
                new_matches_count+=1
    
    print(f"origin matches: {old_matches_count}, kept matches: {new_matches_count}")
    return new_matches

In [16]:
def get_fms(kpts, matches,threshold=5):
    prev_len = dict()
    fms = defaultdict(dict)
    print("Get Fundamental Matrix")
    for key1 in tqdm(matches):
        for key2 in matches[key1]:
            match = np.array(matches[key1][key2])
            mkpts1 = kpts[key1][match[:, 0]]
            mkpts2 = kpts[key2][match[:, 1]]
            #print(mkpts1.shape)
            new_match = match  

            try:
                Fm, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, threshold, FM_PARAMS["confidence"], FM_PARAMS["maxIters"])
                if FM_PARAMS["removeOutliers"] and inliers is not None:
                    new_match = match[inliers.ravel() == 1]
            except Exception as e:
                print(f"Failed to compute fundamental matrix for {key1} and {key2}: {e}")
                Fm = None

            if FM_PARAMS["removeOutliers"] == True:
                matches[key1][key2] = new_match
            fms[key1][key2] = Fm
    # print(Fm.shape)
    return kpts, matches, fms

In [17]:
if MODEL_DICT["Keynet"]["enable"]:
    keynet_model = (
        AffNetHardNet(num_features=8000, upright=False, device=device, detector="keynet")
        .to(device)
        .eval()
    )
    keynet_detector = AffNetHardNetDetector(keynet_model, resize_long_edge_to=MODEL_DICT["Keynet"]["resize_long_edge_to"])
    keynet_detector_1024 = AffNetHardNetDetector(keynet_model, resize_long_edge_to=1024)
    laf_matcher = LafMatcher(device=device)
    
    
if MODEL_DICT["GFTT"]["enable"]:
    gftt_model = (
        AffNetHardNet(num_features=8000, upright=False, device=device, detector="GFTT")
        .to(device)
        .eval()
    )
    gftt_detector = AffNetHardNetDetector(gftt_model, resize_long_edge_to=MODEL_DICT["GFTT"]["resize_long_edge_to"])
    gftt_detector_1024 = AffNetHardNetDetector(gftt_model, resize_long_edge_to=1024)
    laf_matcher = LafMatcher(device=device)

if MODEL_DICT["DoG"]["enable"]:
    DoG_model = (
        AffNetHardNet(num_features=8000, upright=False, device=device, detector="DoG")
        .to(device)
        .eval()
    )
    DoG_detector = AffNetHardNetDetector(DoG_model, resize_long_edge_to=MODEL_DICT["DoG"]["resize_long_edge_to"])
    DoG_detector_1024 = AffNetHardNetDetector(DoG_model, resize_long_edge_to=1024)
    laf_matcher = LafMatcher(device=device)

if MODEL_DICT["Harris"]["enable"]:
    harris_model = (
        AffNetHardNet(num_features=8000, upright=False, device=device, detector="Harris")
        .to(device)
        .eval()
    )
    harris_detector = AffNetHardNetDetector(harris_model, resize_long_edge_to=MODEL_DICT["Harris"]["resize_long_edge_to"])
    harris_detector_1024 = AffNetHardNetDetector(harris_model, resize_long_edge_to=1024)
    laf_matcher = LafMatcher(device=device)
    
if MODEL_DICT["Disk"]["enable"]:
    disk_lightglue_matcher = LafMatcher(matcher="lightglue", device=device)

/tmp/ipykernel_26/1752230650.py:43: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600
Init AffNetHardNetDetector
Longer edge will be resized to 1024


/tmp/ipykernel_26/1752230650.py:55: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600
Init AffNetHardNetDetector
Longer edge will be resized to 1024


/tmp/ipykernel_26/1752230650.py:71: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600
Init AffNetHardNetDetector
Longer edge will be resized to 1024


/tmp/ipykernel_26/1752230650.py:63: DeprecationWarning: `LAFAffNetShapeEstimator` default behaviour is changed and now it does preserve original LAF orientation. Make sure your code accounts for this.
  aff_module=KF.LAFAffNetShapeEstimator(False).eval(),


Init AffNetHardNetDetector
Longer edge will be resized to 1600
Init AffNetHardNetDetector
Longer edge will be resized to 1024


In [18]:
def get_keypoint_matches(file_path, key1, key2):
    with h5py.File(file_path, 'r') as file:
        matches_path = f'{key1}/{key2}/matches0'
        matching_scores_path = f'{key1}/{key2}/matching_scores0'
        
        matches = file[matches_path].__array__()
        matching_scores = file[matching_scores_path].__array__()
        
        valid_indices = np.where((matches != -1) & (matching_scores >= 0.2))[0]
        matched_indices = matches[valid_indices]
        result = np.stack((valid_indices, matched_indices), axis=-1)
        return result

def HLOC(img_fnames,model_name,mode,dataset):
    images = Path(f'/kaggle/input/image-matching-challenge-2024/{mode}/{dataset}/images')
    outputs = Path(f'outputs/{model_name}/{dataset}')
    !rm -rf $outputs
    sfm_pairs = outputs / 'pairs-sfm.txt'
    features = outputs / 'features.h5'
    matches = outputs / 'matches.h5'
    if model_name == "disk":
        feature_conf = extract_features.confs[model_name]
    elif model_name == "superpoint":
        feature_conf = extract_features.confs[f"{model_name}_inloc"]
    matcher_conf = match_features.confs[f'{model_name}+lightglue']
    
    references = [Path(name).name for name in img_fnames]
    
    extract_features.main(feature_conf, images, image_list=references, feature_path=features)
    pairs_from_exhaustive.main(sfm_pairs, image_list=references)
    match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)
    
    kpts = defaultdict()
    with h5py.File(features, 'r') as f:
        for file_name, file_group in f.items():
            keypoints = np.array(file_group['keypoints'])
#             print(keypoints.shape)
            kpts[file_name] = keypoints
    
    _matches = defaultdict(dict)
    num_imgs = len(img_fnames)
    pair_cnt = 0
    for idx1 in tqdm(range(num_imgs - 1)):
        for idx2 in range(idx1 + 1, num_imgs):
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split("/")[-1], fname2.split("/")[-1]
            try:
                tmp = get_keypoint_matches(matches,key1,key2)
                if len(tmp) <= 20: continue
                pair_cnt += 1
                _matches[key1][key2] = tmp
            except Exception as e:
                tmp = get_keypoint_matches(matches,key2,key1)
                if len(tmp) <= 20: continue
                pair_cnt += 1
                _matches[key1][key2] = tmp[...,::-1]
    
    print(f"Get {pair_cnt} from {int(num_imgs * (num_imgs-1)/2)}")

    return kpts, _matches


In [19]:
def splitter(kpts,matches,dataset,img_dir):
    kornia_kpts = deepcopy(kpts)
    kornia_matches = deepcopy(matches)
    kornia_kpts, kornia_matches, kornia_fms = get_fms(kornia_kpts, kornia_matches, 5)
    kornia_matches = select_matches(kornia_matches)
    
     # Write to database
    feature_dir = f"featureout/kornia/{dataset}"
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir, exist_ok=True)
    database_path = f"{feature_dir}/colmap.db"
    if os.path.isfile(database_path):
        os.remove(database_path)

    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    print("Add kpts and matches to database")
    add_kpts_matches(db, img_dir, kornia_kpts, kornia_matches, kornia_fms)
    torch.cuda.empty_cache()
    gc.collect()

    

In [20]:
def generate_scene_db(dataset):
    feature_det_start = time()
    # Process a scene and write matches and keypoints to the database
    img_dir = f"{SRC}/{MODE}/{dataset}/images"
    if not os.path.exists(img_dir):
        print("Image dir does not exist:", img_dir)
        return

    img_fnames = [f"{SRC}/{x}" for x in data_dict[dataset]]
    print(f"Got {len(img_fnames)} images")

    matches = defaultdict(dict)
    kpts = defaultdict()

    if MODEL_DICT["Keynet"]["enable"]:
        print("Keynet kpts and match")
        f_lafs, f_kpts, f_descs, f_raw_size = keynet_detector.detect_features(
            img_fnames
        )
        keynet_pairs, keynet_kpts, keynet_matches, keynet_rois = laf_matcher.match(
            img_fnames, f_lafs, f_kpts, f_descs, f_raw_size
        )
        if not MODEL_DICT["Keynet"]["pair_only"]:
            kpts, matches = merge_kpts_matches(kpts, matches, keynet_kpts, keynet_matches, MATCHES_CAP)
            
        print("Keynet kpts and match - 1024")
        f_lafs, f_kpts, f_descs, f_raw_size = keynet_detector_1024.detect_features(
            img_fnames
        )
        keynet_pairs, keynet_kpts, keynet_matches, keynet_rois = laf_matcher.match(
            img_fnames, f_lafs, f_kpts, f_descs, f_raw_size
        )
        if not MODEL_DICT["Keynet"]["pair_only"]:
            kpts, matches = merge_kpts_matches(kpts, matches, keynet_kpts, keynet_matches, MATCHES_CAP)
            
        del  keynet_pairs, keynet_kpts, keynet_matches, keynet_rois
    
    
    if MODEL_DICT["GFTT"]["enable"]:
        print("GFTT kpts and match")
        gftt_lafs, gftt_kpts, gftt_descs, gftt_raw_size = gftt_detector.detect_features(
            img_fnames
        )
        index_pairs, gftt_kpts, gftt_matches, gftt_rois = laf_matcher.match(
            img_fnames, gftt_lafs, gftt_kpts, gftt_descs, gftt_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, gftt_kpts, gftt_matches, MATCHES_CAP)
        
        print("GFTT kpts and match - 1024")
        gftt_lafs, gftt_kpts, gftt_descs, gftt_raw_size = gftt_detector_1024.detect_features(
            img_fnames
        )
        index_pairs, gftt_kpts, gftt_matches, gftt_rois = laf_matcher.match(
            img_fnames, gftt_lafs, gftt_kpts, gftt_descs, gftt_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, gftt_kpts, gftt_matches, MATCHES_CAP)
        
        del gftt_lafs, gftt_kpts, gftt_descs, gftt_raw_size
        
    if MODEL_DICT["DoG"]["enable"]:
        DoG_lafs, DoG_kpts, DoG_descs, DoG_raw_size = DoG_detector.detect_features(
            img_fnames
        )
        index_pairs, DoG_kpts, DoG_matches, DoG_rois = laf_matcher.match(
            img_fnames, DoG_lafs, DoG_kpts, DoG_descs, DoG_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, DoG_kpts, DoG_matches, MATCHES_CAP)
        
        print("DOG kpts and match - 1024")
        DoG_lafs, DoG_kpts, DoG_descs, DoG_raw_size = DoG_detector_1024.detect_features(
            img_fnames
        )
        index_pairs, DoG_kpts, DoG_matches, DoG_rois = laf_matcher.match(
            img_fnames, DoG_lafs, DoG_kpts, DoG_descs, DoG_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, DoG_kpts, DoG_matches, MATCHES_CAP)
        
        del DoG_lafs, DoG_kpts, DoG_descs, DoG_raw_size
        
    if MODEL_DICT["Harris"]["enable"]:
        harris_lafs, harris_kpts, harris_descs, harris_raw_size = harris_detector.detect_features(
            img_fnames
        )
        harris_pairs, harris_kpts, harris_matches, harris_rois = laf_matcher.match(
            img_fnames, harris_lafs, harris_kpts, harris_descs, harris_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, harris_kpts, harris_matches, MATCHES_CAP)
        
        print("Harris kpts and match - 1024")
        harris_lafs, harris_kpts, harris_descs, harris_raw_size = harris_detector_1024.detect_features(
            img_fnames
        )
        harris_pairs, harris_kpts, harris_matches, harris_rois = laf_matcher.match(
            img_fnames, harris_lafs, harris_kpts, harris_descs, harris_raw_size
        )
        kpts, matches = merge_kpts_matches(kpts, matches, harris_kpts, harris_matches, MATCHES_CAP)
        
        del harris_pairs, harris_kpts, harris_matches, harris_rois
    
    if SPLIT: splitter(kpts,matches,dataset,img_dir)

    if MODEL_DICT["Disk"]["enable"]:
        disk_kpts, disk_matches = HLOC(img_fnames,"disk",MODE,dataset)
        kpts, matches = merge_kpts_matches(kpts, matches, disk_kpts, disk_matches, MATCHES_CAP)
        del disk_kpts, disk_matches

        
    if MODEL_DICT["Superpoint"]["enable"]:
        superpoint_kpts, superpoint_matches = HLOC(img_fnames,"superpoint",MODE,dataset)
        kpts, matches = merge_kpts_matches(kpts, matches, superpoint_kpts, superpoint_matches, MATCHES_CAP)
        del superpoint_kpts, superpoint_matches
        
#     print(kpts['00001.png'])
#     print(matches['00001.png'])
    # if MODEL_DICT["RoMa"]["enable"]:
    #     RoMa_kpts, RoMa_matches, RoMa_fms = RoMa(img_fnames,kpts,device)
    #     kpts, matches = merge_kpts_matches(kpts, matches, RoMa_kpts, RoMa_matches, MATCHES_CAP)
        # kpts,matches,fms = RoMa_kpts, RoMa_matches, RoMa_fms
        # del RoMa_kpts, RoMa_matches
        # gc.collect()
        
    # Get fundamental matrices
    kpts, matches, fms = get_fms(kpts, matches, 5)
    matches = select_matches(matches, MATCH_FILTER_RATIO)
    # print(len(kpts))
    # print(len(matches))
    '''
        index_pairs[image1_index][image2_index, dist_mean, num_matches]
        kpts['0001.png'] = (number of keypoints, coordinates)
        matches['0001.png']['0006.png'] = (num_matched_points, 2) <= two indices in keypts
    '''

    

    # for _ in range(5):
    #     print(matches.keys())
    #     key1 = random.choice(list(matches.keys()))
    #     key2 = random.choice(list(matches[key1].keys()))
    #     print(key1, key2)
    #     fname1, fname2 = os.path.join(img_dir, key1), os.path.join(img_dir, key2)

    #     print("Plot Combined matches")
    #     plot_images_with_keypoints(
    #         fname1, fname2, kpts[key1], kpts[key2], matches[key1][key2]
    #     )
        
    # Write to database
    feature_dir = f"featureout/KDS/{dataset}"
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir, exist_ok=True)
    database_path = f"{feature_dir}/colmap.db"
    if os.path.isfile(database_path):
        os.remove(database_path)


    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    print("Add kpts and matches to database")
    add_kpts_matches(db, img_dir, kpts, matches, fms)
    feature_det_end = time()
    matching_time = feature_det_end - feature_det_start
    torch.cuda.empty_cache()
    gc.collect()

    return matching_time

In [21]:
def reconstruct_from_db(dataset, type):
    scene_result = {}
    reconst_start = time()

    img_dir = f"{SRC}/{MODE}/{dataset}/images"
    if not os.path.exists(img_dir):
        print("Image dir does not exist:", img_dir)
        return

    feature_dir = f"featureout/{type}/{dataset}"
    database_path = f"{feature_dir}/colmap.db"
    db = COLMAPDatabase.connect(database_path)
    output_path = f"{feature_dir}/colmap_rec"
    t = time()
    gc.collect()

#    # Skip match_exhaustive
#     pycolmap.match_exhaustive(database_path, match_options)

    t = time() - t
    print(f"RANSAC in  {t:.4f} sec")
    t = time()
    
    # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
    # pipeline_option = pycolmap.IncrementalPipelineOptions()
    
    
    # mapper_option=pycolmap.IncrementalMapperOptions()
    # mapper_option.min_model_size = 3
    # os.makedirs(output_path, exist_ok=True)
    # maps = pycolmap.incremental_mapping(
    #     database_path=database_path,
    #     image_path=img_dir,
    #     output_path=output_path,
    #     options = mapper_option
    # )
    
    mapper_options = pycolmap.IncrementalMapperOptions(min_model_size = 3)
    maps = pycolmap.incremental_mapping(
                    database_path=database_path, 
                    image_path=img_dir,
                    output_path=output_path, 
                    options=mapper_options,
                )
    
    print(maps)
    # clear_output(wait=False)
    t = time() - t
    print(f"Reconstruction done in  {t:.4f} sec")
    imgs_registered = 0
    best_idx = None
    num_points = 0
    print("Looking for the best reconstruction")
    if isinstance(maps, dict):
        for idx1, rec in maps.items():
            print(idx1, rec.summary())
            if len(rec.images) > imgs_registered:
                imgs_registered = len(rec.images)
                best_idx = idx1
                num_points = len(rec.points3D)
    if best_idx is not None:
        print(maps[best_idx].summary())
        for k, im in maps[best_idx].images.items():
            key1 = f"{MODE}/{dataset}/images/{im.name}"
            scene_result[key1] = {}
#             scene_result[key1]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
#             scene_result[key1]["t"] = deepcopy(np.array(im.cam_from_world.translation))
            scene_result[key1]["R"] = deepcopy(im.rotmat())
            scene_result[key1]["t"] = deepcopy(np.array(im.tvec))

    # fig = viz_3d.init_figure()
    # viz_3d.plot_reconstruction(fig, maps[best_idx], color='rgba(255,0,0,0.5)', name="mapping", points_rgb=True)
    # fig.show()
    
    gc.collect()
    reconst_end = time()
    reconst_time = reconst_end - reconst_start
    return scene_result, reconst_time, best_idx, num_points

In [22]:
# Main loop to add kpts and matches
datasets = []
time_dict = dict()
best_models = defaultdict(dict)
num_points = defaultdict(dict)

for dataset in data_dict:
    datasets.append(dataset)

if DEBUG:
    matching_start = time()
    for dataset in all_datasets:
        print(dataset)
        time_dict["matching-" + dataset] = generate_scene_db(dataset)

    matching_end = time()
    time_dict["matching-TOTAL"] = matching_end - matching_start
else:
    # Run db generation and reconstuction with multiprocessing if not DEBUG
    out_results = defaultdict(dict)
    total_start = time()
    with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_CORES) as executors:
        futures = defaultdict(dict)

        for dataset in all_datasets:
            print(dataset)
            time_dict["matching-" + dataset] = generate_scene_db(dataset)
            futures[dataset]['KDS'] = executors.submit(reconstruct_from_db, dataset,"KDS") #kornia*4 + disk + superpoint
            if SPLIT: futures[dataset]['kornia'] = executors.submit(reconstruct_from_db, dataset,"kornia")
            

        for dataset in all_datasets:
            result_kds = futures[dataset]["KDS"].result()
            if SPLIT: result_kornia = futures[dataset]["kornia"].result()
            if result_kds is not None:
                out_results[dataset]["KDS"], time_dict["reconst-" + dataset], best_models[dataset]['KDS'], num_points[dataset]['KDS'] = result_kds
            if SPLIT and result_kornia is not None:
                out_results[dataset]["kornia"], time_dict["reconst-" + dataset], best_models[dataset]["kornia"], num_points[dataset]["kornia"] = result_kornia
                
    total_end = time()
    time_dict["TOTAL"] = total_end - total_start

church
Got 41 images
Keynet kpts and match
Detecting AffNetHardNet features


100%|██████████| 41/41 [00:20<00:00,  2.03it/s]


Matching to get index pairs


100%|██████████| 40/40 [00:33<00:00,  1.18it/s]


 Get 233 from 820 possible pairs
Keynet kpts and match - 1024
Detecting AffNetHardNet features


...

  45  9.910545e+04    6.53e-03    2.52e+05   8.07e+05   1.00e+00  3.04e+08        1    2.85e-01    1.29e+01
  46  9.910545e+04    0.00e+00    2.52e+05   0.00e+00   0.00e+00  1.52e+08        1    1.78e-01    1.31e+01


W20240603 14:06:52.030103   132 levenberg_marquardt_strategy.cc:115] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.
W20240603 14:06:52.203755   132 levenberg_marquardt_strategy.cc:115] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.


  47  9.910545e+04    0.00e+00    2.52e+05   0.00e+00   0.00e+00  3.80e+07        1    1.74e-01    1.33e+01
  48  9.910545e+04    2.42e-03    1.49e+04   3.02e+05   1.00e+00  1.14e+08        1    2.93e-01    1.36e+01
  49  9.910545e+04    0.00e+00    1.49e+04   0.00e+00   0.00e+00  5.70e+07        1    1.77e-01    1.37e+01


W20240603 14:06:52.674021   132 levenberg_marquardt_strategy.cc:115] Linear solver failure. Failed to compute a step: Eigen failure. Unable to perform dense Cholesky factorization.


  50  9.910544e+04    3.58e-03    2.24e+03   4.50e+05   1.00e+00  1.71e+08        1    2.86e-01    1.40e+01


Bundle adjustment report
------------------------
    Residuals : 263434
   Parameters : 86023
   Iterations : 51
         Time : 14.0385 [s]
 Initial cost : 0.621819 [px]
   Final cost : 0.613356 [px]
  Termination : No convergence

  => Completed observations: 257
  => Merged observations: 378
  => Filtered observations: 163
  => Changed observations: 0.006058

Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  9.967523e+04    0.00e+00    6.12e+04   0.00e+00   0.00e+00  1.00e+04        0    9.43e-02    5.49e-01
   1  9.843678e+04    1.24e+03    4.67e+03   2.98e+02   9.99e-01  3.00e+04        1    3.24e-01    8.73e-01
   2  9.840958e+04    2.72e+01    4.92e+03   4.03e+03   9.99e-01  9.00e+04        1    3.01e-01    1.17e+00
   3  9.839098e+04    1.86e+01    1.42e+04   8.24e+03   9.28e-01  

In [23]:
# Main loop for reconstruction
if DEBUG:
    reconst_start = time()
    best_models = defaultdict(dict)
    out_results = defaultdict(dict)
    num_points = defaultdict(dict)
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=NUM_CORES) as executors:
        
        futures = defaultdict(dict)

        for dataset in all_datasets:
            print(dataset)
            futures[dataset]['KDS'] = executors.submit(reconstruct_from_db, dataset,"KDS") #kornia*4 + disk + superpoint
            if SPLIT: futures[dataset]['kornia'] = executors.submit(reconstruct_from_db, dataset,"kornia")
            

        for dataset in all_datasets:
            result_kds = futures[dataset]["KDS"].result()
            if SPLIT: result_kornia = futures[dataset]["kornia"].result()
            if result_kds is not None:
                out_results[dataset]["KDS"], time_dict["reconst-" + dataset], best_models[dataset]['KDS'], num_points[dataset]['KDS'] = result_kds
            if SPLIT and result_kornia is not None:
                out_results[dataset]["kornia"], time_dict["reconst-" + dataset], best_models[dataset]["kornia"], num_points[dataset]["kornia"] = result_kornia
                
    reconst_end = time()
    time_dict["reconst-TOTAL"] = reconst_end - reconst_start

In [24]:


# def quaternion_to_rotation_matrix(q):
#     # Normalize the quaternion
#     q = q / np.linalg.norm(q)
#     q0, q1, q2, q3 = q[3], q[0], q[1], q[2]  # assuming input q is in the form [x, y, z, w]
    
#     # Compute the rotation matrix
#     R = np.array([
#         [1 - 2*(q2**2 + q3**2), 2*(q1*q2 - q0*q3),     2*(q1*q3 + q0*q2)],
#         [2*(q1*q2 + q0*q3),     1 - 2*(q1**2 + q3**2), 2*(q2*q3 - q0*q1)],
#         [2*(q1*q3 - q0*q2),     2*(q2*q3 + q0*q1),     1 - 2*(q1**2 + q2**2)]
#     ])
    
#     return R

# model_name = "superpoint"
# for dataset in all_datasets:
#     outputs = Path(f'outputs/{model_name}/{dataset}')
#     features = outputs / 'features.h5'
#     matches = outputs / 'matches.h5'
#     key_names = [x.split('/')[-1] for x in data_dict[dataset]]
#     try:
#         model = pycolmap.Reconstruction(f"/kaggle/working/featureout/{dataset}/colmap_rec/{best_models[dataset]}")      
#     except Exception as e:
#         print(f"Load {dataset} model failed: {e}")  
#         continue
#     ref_ids = [model.find_image_with_name(r).image_id for r in key_names if model.find_image_with_name(r) is not None]
#     for key in key_names:
#         if model.find_image_with_name(key): continue
#         camera = pycolmap.infer_camera_from_image(f"{SRC}/{MODE}/{dataset}/images/{key}")
#         conf = {
#             'estimation': {'ransac': {'max_error': 12}},
#             'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
#         }
#         localizer = QueryLocalizer(model, conf)
#         try:
#             ret, log = pose_from_cluster(localizer, key, camera, ref_ids, features, matches)
#             rotmax = quaternion_to_rotation_matrix(ret['qvec'])
#             out_results[dataset][key] = {}
#             out_results[dataset][key]["R"] = rotmax
#             out_results[dataset][key]["t"] = ret['tvec']
#         except Exception as e:
#             print(f"Error processing {key} in dataset {dataset}: {e}")
        
        

In [25]:
from time import sleep

final_result = defaultdict()
for dataset in data_dict:
    print(f"{dataset} --- {len(out_results[dataset]['KDS'])} / {len(data_dict[dataset])}")
    print(f"number of 3d points: {num_points[dataset]['KDS']}")
    final_result[dataset] = out_results[dataset]['KDS']
    if SPLIT:
        print(f"{dataset} --- {len(out_results[dataset]['kornia'])} / {len(data_dict[dataset])}")
        print(f"number of 3d points: {num_points[dataset]['kornia']}")
        if num_points[dataset]['kornia'] > num_points[dataset]['KDS'] or len(out_results[dataset]['kornia']) >= len(out_results[dataset]['KDS'])-1:
            print("choose Kornia")
            final_result[dataset] = out_results[dataset]['kornia']
        else: sleep(900)
        
create_submission(final_result, data_dict, MODE)

church --- 32 / 41
number of 3d points: 61056
church --- 37 / 41
number of 3d points: 77088
choose Kornia
test/church/images/00046.png
test/church/images/00090.png
test/church/images/00092.png
test/church/images/00087.png
test/church/images/00050.png
test/church/images/00068.png
test/church/images/00083.png
test/church/images/00096.png
test/church/images/00069.png
test/church/images/00081.png
test/church/images/00042.png
test/church/images/00018.png
test/church/images/00030.png
test/church/images/00024.png
test/church/images/00032.png
test/church/images/00026.png
test/church/images/00037.png
test/church/images/00008.png
test/church/images/00035.png
test/church/images/00021.png
test/church/images/00010.png
test/church/images/00039.png
test/church/images/00011.png
test/church/images/00013.png
test/church/images/00006.png
test/church/images/00012.png
test/church/images/00029.png
test/church/images/00001.png
test/church/images/00072.png
test/church/images/00066.png
test/church/images/00059